#Task overview

The aim of this competition is to predict 3-commerce clicks, cart , orders. 

You 'll build a multi-objective rec-sys based on previous events in a user session.

Current recommender syustems consist of various models with different approaches, ranging from simple matrix factorization to a transformer-tyoe deeo neural network.

however, no single model exists that can simultaneously optimize multiple objectives.

In this competition, you'll build a single entry to predict click-through, add to cart, and conversion rates based on previous same-session events.

#Imports/setup

In [ ]:
###imports###

import pandas as pd
from pathlib import Path

import os
import random
import numpy as np
import json

from datetime import timedelta
from collections import Counter
from tqdm.notebook import tqdm
from heapq import nlargest

import matplitlib.pyplot as plt
import seaborn as sns
sns.set_theme()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
## PATHS###

data_path = Path('../input/otto-recommender-system')
train_path = Path(data_path/'train.json')
test_path = Path(data_path/'test.json')

sample_sub_path = Path('../input/otto-recommender-system/sample_submission.csv')

#Load in the data

In [ ]:
#lets check how many lines the trainin gdata has

with open(train_path, 'r') as f :
  print(f'we have {len(f.readlines()):,} lines in the training data')

In [ ]:
# Load in a sample to a pandas df

sample_size = 150000

chunkcs = pd.read_json(train_path, lines=True, chunksize=sample_size)

for c in chunks :
  sample_train_df = c
  break

In [ ]:
sample_train_df.set_index('session', drop=True, inplace=True)

sample_train_df.head()

#Data Structure

session : the unique session id. Each session contains a list of time ordered events.

events : the time ordered sequence of events in the session. Each event contains 3 pieces of information :

- aid : the article di of the associated event
- ts  : the unix timestamp of the event.
- type : the event type, i.e., whether a product was clicked(clikcs), added to the user's cart, or ordered during the session

In [ ]:
# Let's look at an example ssession and print out some basic info

# Sample the first session in the df
example_session = sample_train_df.iloc[0].item()
print(f'This session was {len(example_session)} actions long \n')
print(f'The first action in the session : \n {example_session[0]}')

# Time of session
time_elapsed = example_session[-1]['ts'] - example_session[0]['ts']

# The timestamp is in milliseconds since 00 00 00 utc on 1 jan 1970
print(f'The first session elapsed : {str(timedelta(milliseconds=time_elapsed))} \n')

# Count the freq of actions within the session

action_counts = {}

for action in example_session :
  action_counts[action['type']] = action_counts.get(action['type'], 0 ) +1

print(f'The first session contains the following freq of actions : {action_counts}')

# Initial EDA

In [ ]:
### Extract information from each session and add it to the df ###

action_counts_list, article_id_counts_list, session_length_time_list, session_lengtth_action_list = [[] for _ in range(4)]

overall_action_counts = {}
overall_article_id_counts= {}

for i,row in tqdm(sample_train_df.iterrows(), total=len(sample_train_df)) :
  actions = row['events']

  #Get the freq of actions and article_ids
  action_counts = {}
  article_id_counts = {}

  for action in actions :
    action_counts[action['type']] = action_counts.get(action['type'],0) +1

    article_id_counts[action['aid']] = aciton_counts.get(action['aid'],0) +1

    overall_action_counts[action['type']] = overall_action_counts.get(action['type'],0) +1
    overall_article_id_counts[action['aid']] = overall_article_id_counts.get(action['aid'],0) +1

  
  # GEt the length of the session
  session_length_time = actions[-1]['ts'] - actions[0]['ts']


  # ADd to list
  action_counts_list.append(action_counts)
  article_id_counts_list.append(article_id_counts)

  session_length_action_list.append(len(actions))


sample_train_df['action_counts'] = action_counts_list
sample_train_df['article_id_counts'] = article_id_counts_list

sample_train_df['session_length_unix'] = session_length_time_list
sample_train_df['session_length_hours'] = sample_train_df['session_length_unix']*2.77778e-7

sample_train_df['session_length_action'] = session_length_action_list

In [ ]:
### Actions ###

total_actions = sum(overall_action_counts.values())

plt.figure(figsize=(8,6))
sns.barplot(list(overall_action_counts.keys()), [i/total_actions for i in overall_actions_counts.values()]);

plt.title(f'Action freq', fontsize=12)
plt.xlabel('Count')
plt.ylabel('cat')
plt.show()

